In [1]:
!pip install yfinance ta backtesting --quiet

In [2]:
import yfinance as yf
import pandas as pd
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import matplotlib.pyplot as plt

/Users/michaelkovacevich/.venv/lib/python3.7/site-packages/urllib3/__init__.py:38: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  exceptions.NotOpenSSLWarning,
/Users/michaelkovacevich/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_intraday_data(ticker: str, period: str = "7d", interval: str = "1m") -> pd.DataFrame:
    df = yf.download(ticker, period=period, interval=interval)
    df = df.dropna()
    return df

def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    df["SMA_20"] = ta.trend.sma_indicator(df["Close"], window=20)
    df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
    df["RSI"] = ta.momentum.rsi(df["Close"], window=14)
    df["MACD"] = ta.trend.macd_diff(df["Close"])
    return df

class SmaCross(Strategy):
    def init(self):
        self.sma20 = self.I(lambda x: pd.Series(x).rolling(20).mean(), self.data.Close)
        self.sma50 = self.I(lambda x: pd.Series(x).rolling(50).mean(), self.data.Close)

    def next(self):
        if crossover(self.sma20, self.sma50):
            self.buy()
        elif crossover(self.sma50, self.sma20):
            self.sell()

class RsiStrategy(Strategy):
    def init(self):
        self.rsi = self.I(ta.momentum.RSIIndicator, self.data.Close, 14).rsi()

    def next(self):
        if self.rsi[-1] < 30:
            self.buy()
        elif self.rsi[-1] > 70:
            self.sell()

class MacdStrategy(Strategy):
    def init(self):
        self.macd = self.I(ta.trend.macd_diff, self.data.Close)

    def next(self):
        if self.macd[-1] > 0 and self.macd[-2] <= 0:
            self.buy()
        elif self.macd[-1] < 0 and self.macd[-2] >= 0:
            self.sell()

In [4]:
def run_backtest(df: pd.DataFrame, strategy_cls: Strategy):
    df_bt = df.copy()
    df_bt = df_bt[['Open', 'High', 'Low', 'Close', 'Volume']]
    df_bt.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    bt = Backtest(df_bt, strategy_cls, cash=10_000, commission=0.002)
    stats = bt.run()
    equity = bt._equity_curve["Equity"].reset_index(drop=True)
    return stats, equity

In [5]:
ticker = "AAL"
df = load_intraday_data(ticker)
df = add_indicators(df)

sma_stats, sma_equity = run_backtest(df, SmaCross)
rsi_stats, rsi_equity = run_backtest(df, RsiStrategy)
macd_stats, macd_equity = run_backtest(df, MacdStrategy)

min_len = min(len(sma_equity), len(rsi_equity), len(macd_equity))
sma_equity = sma_equity[:min_len]
rsi_equity = rsi_equity[:min_len]
macd_equity = macd_equity[:min_len]

plt.figure(figsize=(12, 6))
plt.plot(sma_equity, label="SMA Crossover")
plt.plot(rsi_equity, label="RSI Strategy")
plt.plot(macd_equity, label="MACD Strategy")
plt.title("Equity Curve Comparison")
plt.xlabel("Trade #")
plt.ylabel("Equity")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().